In [141]:
#import BeautifulSoup
from bs4 import BeautifulSoup

#import re module for REGEXes
import re

#import pandas
import pandas as pd

#import extractor if all else fails
from sec_api import ExtractorApi

#open the actual file
with open("rando.html") as f:
        page = f.read()



In [142]:
print(len(page))

5982811


In [143]:
# Regex to find <DOCUMENT> tags
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')

In [144]:
# Create 3 lists with the span idices for each regex

### There are many <Document> Tags in this text file, each as specific exhibit like 10-K, EX-10.17 etc
### First filter will give us document tag start <end> and document tag end's <start> 
### We will use this to later grab content in between these tags
doc_start_is = [x.end() for x in doc_start_pattern.finditer(page)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(page)]

print(doc_start_is)
print(doc_end_is)

### Type filter is interesting, it looks for <TYPE> with Not flag as new line, ie terminate there, with + sign
### to look for any char afterwards until new line \n. This will give us <TYPE> followed Section Name like '10-K'
### Once we have have this, it returns String Array, below line will with find content after <TYPE> ie, '10-K' 
### as section names
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(page)]

print(doc_types)

[1196, 860734, 861779, 862786, 879959, 894732, 909532, 914161, 918799, 1289469, 1336065, 1380838, 1500955, 1739721, 1933927, 4059673, 4378495, 4730681, 4771382, 4841712, 4864549, 4920588, 4942556, 5002305, 5020268, 5042428, 5051519, 5056962, 5062036, 5070712, 5078767, 5084662, 5098805, 5109953, 5148800, 5157381, 5180003, 5198862, 5204562, 5251687, 5259940, 5268157, 5274910, 5282873, 5303883, 5316656, 5339431, 5349735, 5354478, 5360773, 5368216, 5377034, 5386890, 5405052, 5411586, 5450648, 5459837, 5481677, 5495208, 5514336, 5527047, 5532132, 5625361, 5667596, 5719862, 5726350, 5829375, 5830460, 5833261, 5862704]
[860712, 861757, 862764, 879937, 894710, 909510, 914139, 918777, 1289447, 1336043, 1380816, 1500933, 1739699, 1933905, 4059651, 4378473, 4730659, 4771360, 4841690, 4864527, 4920566, 4942534, 5002283, 5020246, 5042406, 5051497, 5056940, 5062014, 5070690, 5078745, 5084640, 5098783, 5109931, 5148778, 5157359, 5179981, 5198840, 5204540, 5251665, 5259918, 5268135, 5274888, 5282851, 

Create a Dictionary for the 10-K

In the code below, we will create a dictionary which has the key 10-K and as value the contents of the 10-K section found above. To do this, we will create a loop, to go through all the sections found above, and if the section type is 10-K then save it to the dictionary. Use the indices in  doc_start_is and doc_end_isto slice the raw_10k file.

In [145]:
document = {}

# Create a loop to go through each section type and save only the 10-K section in the dictionary
for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K':
        document[doc_type] = page[doc_start:doc_end]

In [146]:
# display excerpt the document
document['10-K'][0:500]

'\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>form10k.htm\n<DESCRIPTION>10-K\n<TEXT>\n<html>\n  <head>\n    <title></title>\n    <!-- Licensed to: The Ideal Connection\n         Document created using EDGARfilings PROfile 7.0.1.0\n         Copyright 1995 - 2020 Broadridge -->\n  </head>\n<body text="#000000" bgcolor="#ffffff" style="FONT-SIZE: 10pt; FONT-FAMILY: \'Times New Roman\', Times, serif">\n  <div>\n    <div style="text-align: left;">\n      <hr align="center" style="BORDER-TOP: black 4px solid; HEIGHT: 10px; BORD'

In [147]:
# Write the regex
regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(7A|7)\.{0,1})|(ITEM\s(7A|7))')

# Use finditer to math the regex
matches = regex.finditer(document['10-K'])

# Write a for loop to print the matches
for match in matches:
    print(match)

<re.Match object; span=(167598, 167604), match='ITEM 7'>
<re.Match object; span=(220854, 220861), match='ITEM 7A'>


In [148]:
# Matches
matches = regex.finditer(document['10-K'])

# Create the dataframe
test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

test_df.columns = ['item', 'start', 'end']
test_df['item'] = test_df.item.str.lower()

# Display the dataframe
test_df.head(15)

,item,start,end
0,item 7,167598,167604
1,item 7a,220854,220861


In [149]:
# Get rid of unnesesary charcters from the dataframe
test_df.replace('&#160;',' ',regex=True,inplace=True)
test_df.replace('&nbsp;',' ',regex=True,inplace=True)
test_df.replace(' ','',regex=True,inplace=True)
test_df.replace('\.','',regex=True,inplace=True)
test_df.replace('>','',regex=True,inplace=True)

if len(test_df.index > 2):
    max_len = 0
    end_index = 0
    start = 0

    for row in test_df.itertuples():
        # if this elt is item7a and the previous elt was item7 (defined by "start" not being 0)
        if row[1] == "item7a" and start != 0:
            # then if the length of this section is the greatest so far
            if (row[3] - start) > max_len:
                # set this section to be the new max_len
                max_len = row[3] - start
                # remember the index of this section
                end_index = row[0]
        elif row[1] == "item7":
            start = row[2]
        # if the previous elt was a 7a and the current elt is not an item7, then set current start to 0
        else:
            start = 0
    
    # keep only the indices with the greatest distance between start and end
    test_df = test_df.iloc[end_index-1:end_index+1]

# display the dataframe
test_df.head(15)



,item,start,end
0,item7,167598,167604
1,item7a,220854,220861


Possible loop code if start alone doesn't work

test_df["diff"] = test_df["start"].diff()

max_len = 0
end_index = 0
start = 0

for row in test_df.itertuples():
    # if this elt is item7a and the previous elt was item7 (defined by "start" not being 0)
    if row[1] == "item7a" and start != 0:
        # then if the length of this section is the greatest so far
        if (row[3] - start) > max_len:
            # set this section to be the new max_len
            max_len = row[3] - start
            # remember the index of this section
            end_index = row[0]
    elif row[1] == "item7":
        start = row[2]
    else:
        start = 0

In [150]:
# Drop duplicates
pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='first')

# Display the dataframe
pos_dat

,item,start,end
0,item7,167598,167604
1,item7a,220854,220861


In [151]:
# Set item as the dataframe index
pos_dat.set_index('item', inplace=True)

# display the dataframe
pos_dat

,start,end
item,,
item7,167598,167604
item7a,220854,220861


In [152]:
# Get Item 7
item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item7a']]

In [153]:
### First convert the raw text we have to exrtacted to BeautifulSoup object 
item_7_content = BeautifulSoup(item_7_raw, 'lxml')

In [154]:
print(item_7_content.prettify()[0:1000])

<html>
 <body>
  <p>
   ITEM 7.
  </p>
  <td style="width: 89%; vertical-align: top;">
   <div style="font-weight: bold;">
    MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS
   </div>
  </td>
  <div>
   <br/>
  </div>
  <div style="text-align: justify;">
   <font style="background-color: #FFFFFF;">
    The following discussion should be read in conjunction with the Company's audited consolidated financial statements and the related notes for the year ended January 31, 2020
          and 2019, that appear elsewhere in this annual report. The following discussion contains forward-looking statements that reflect the Company's plans, estimates and beliefs. The Company's actual results could differ materially from those discussed
          in the forward-looking statements. Factors that could cause or contribute to such differences include but are not limited to those discussed below and elsewhere in this annual report.
   </font>
  </div>
  <div>
   <

In [155]:
item_7_text = item_7_content.get_text("\n\n")

In [156]:
print(item_7_text)

ITEM 7.
            
            




MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS













The following discussion should be read in conjunction with the Company's audited consolidated financial statements and the related notes for the year ended January 31, 2020
          and 2019, that appear elsewhere in this annual report. The following discussion contains forward-looking statements that reflect the Company's plans, estimates and beliefs. The Company's actual results could differ materially from those discussed
          in the forward-looking statements. Factors that could cause or contribute to such differences include but are not limited to those discussed below and elsewhere in this annual report.










The Company's consolidated financial statements are stated in United States Dollars and are prepared in accordance with United States Generally Accepted Accounting Principles.




 







Overview of Current Operations




 




Possible API to use? https://sec-api.io/docs/sec-filings-item-extraction-api

In [13]:
extractorApi = ExtractorApi("40f7199296c1efb8083d869d875c033a18ae4224ae774f07a44f1fc61f2f1977")

# Tesla 10-K filing
filing_url = "https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/aapl-20210925.htm"


# get the standardized and cleaned text of section 1A "Risk Factors"
section_html = extractorApi.get_section(filing_url, "7", "html")

# get the original HTML of section 7 "Management’s Discussion and Analysis of Financial Condition and Results of Operations"
section_text = extractorApi.get_section(filing_url, "7", "text")

print(section_text)
#print(section_html)

Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations 

The following discussion should be read in conjunction with the consolidated financial statements and accompanying notes included in Part II, Item 8 of this Form 10-K. This section of this Form 10-K generally discusses 2021 and 2020 items and year-to-year comparisons between 2021 and 2020. Discussions of 2019 items and year-to-year comparisons between 2020 and 2019 are not included in this Form 10-K, and can be found in "Management’s Discussion and Analysis of Financial Condition and Results of Operations" in Part II, Item 7 of the Company’s Annual Report on Form 10-K for the fiscal year ended September 26, 2020. 

Fiscal Year Highlights 

COVID-19 Update 

The COVID-19 pandemic has had, and continues to have, a significant impact around the world, prompting governments and businesses to take unprecedented measures, such as restrictions on travel and business operations, temporary closures o